In [41]:
import yaml
import json
import os
# see: https://github.com/fbrandt/ACLPP/blob/master/doc/format.md
# Put the yaml files in the input directory (scenario: base)


INPUT_PATH = './Data/Data_1/0_Input'
OUTPUT_PATH = './Data/Data_1/1_AeULDs/Ulds_scenario_1'

In [42]:
def createOutputFolderIfNotExists():
    isExist = os.path.exists(OUTPUT_PATH)
    if not isExist:
       # Create a new directory because it does not exist
       os.makedirs(OUTPUT_PATH)
       print("The new directory is created!")

In [43]:
def readYaml(filePathImport):
    with open(filePathImport, "r") as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)

In [44]:
def readJson(filePathImport):
    #Read file in
    with open(filePathImport) as json_file:
        json_file = json.load(json_file)
        return json_file

In [45]:
def writeJson(filePathExport, jsonFile):
    with open(filePathExport, 'w', encoding='utf-8') as f:
        json.dump(jsonFile, f, ensure_ascii=False, indent=4)

In [46]:
def createItemLabels(baseLabel, labelDic):
    appendix = 0
    if baseLabel in labelDic.keys():
        appendix = labelDic[baseLabel] + 1
    
    labelDic[baseLabel] = appendix
    
    return baseLabel + "-" + str(appendix)

In [47]:
def getWeightFromShipment(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return shipments[shipLabel]['pieces'][piece]['weight']

In [48]:
def getRotationAxes(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    
    # rotation axes is unsed
    switcher = {
        1: "NONE",
        2: "X",
        4: "Y",
        5: "Y",
        8: "Z",
        16: "XZ",
        32: "YZ",
        63: "XYZ",
    }
    
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return switcher.get(shipments[shipLabel]['pieces'][piece]['allowed_rotations'], 'NONE')

In [49]:
def getLoadCapacity(sourceItem, shipments):
    shipLabel = sourceItem['shipment']
    piece = sourceItem['piece']
    if shipLabel in shipments.keys():
        if piece in shipments[shipLabel]['pieces'].keys():
            return shipments[shipLabel]['pieces'][piece]['stack_height'] * sourceItem['lat'] * sourceItem['lng'] 
    

In [50]:
def enrichItem(sourceItem, item, sequence, labelDic, shipments):
    item['itemLabel'] = createItemLabels(sourceItem['piece'], labelDic)
    shape = {
        'shapeType' : 'Box',
        # Lat according to Brandt is the width in our system
        'width': sourceItem['lat'],
        'height': sourceItem['height'],
        'depth': sourceItem['lng'],
        'rotationState': 0,
        'id': 2025, # unused
        'volume': sourceItem['lat'] * sourceItem['height'] * sourceItem['lng']
    }
    item['shape'] = shape
    item['x'] = sourceItem['start_lat']
    item['z'] = sourceItem['start_lng']
    item['y'] = sourceItem['start_height']
    item['shipmentLabel'] = sourceItem['shipment']
    
    if item['shipmentLabel'] == "":
        print(item['itemLabel'])
        raise(False)
    
    item['sequence'] = sequence
    item['loadCapacity'] = getLoadCapacity(sourceItem, shipments)
    item['rotationAxes'] = getRotationAxes(sourceItem, shipments)
    item['weight'] = getWeightFromShipment(sourceItem, shipments)
    
    return item

In [51]:
def readUldProperties(sourceUldType):
    mapping = readJson("./01_masterdata/mapping_uld_properties.json")
    if sourceUldType in mapping.keys():
        uldTypeName = mapping[sourceUldType]
        return readJson("./01_masterdata/" + uldTypeName + ".json") 
    else:
        
        sourceUldTypeTrim = sourceUldType[:sourceUldType.rfind("_")]
        
        if sourceUldTypeTrim in mapping.keys():
            uldTypeName = mapping[sourceUldTypeTrim]
            return readJson("./01_masterdata/" + uldTypeName + ".json")
        
        else:
            print(sourceUldType)


In [52]:
def createULD(sourceUld, labelDic, shipments):
    newUld = {"placedItems":[], "id":1}
    newUld["properties"] = readUldProperties(sourceUld["uld_type"])["properties"]
    i = 0
    for sourceItem in sourceUld['loaded']:
        item = readJson('./01_masterdata/placeholders_placed_item.json')
        item = enrichItem(sourceItem, item, i, labelDic, shipments)
        i = i + 1
        newUld['placedItems'].append(item)
    return newUld

In [53]:
## run through flights
def transformFile(sourceFile):
    sourceFile = readYaml(sourceFile)
    for flightKey, segment in sourceFile['segments'].items():
        shipments = segment['shipments']
        for uldKey, sourceUld in segment['built_ulds'].items():
            path_output_full = OUTPUT_PATH + '/' + flightKey + "_" + uldKey + ".json"
            labelDic = {}
            newUld = createULD(sourceUld, labelDic, shipments)
            writeJson(path_output_full, newUld)

In [54]:
def execute():
    createOutputFolderIfNotExists()

    i=0
    for f in os.listdir(INPUT_PATH):
        transformFile(INPUT_PATH + "/" + f)
        print(i)
        i=i+1

In [55]:
execute()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
